# (2주차) 9월9일 
> Path, 이미지 크롤링을 통한 CNN모델 

- toc:true
- branch: master
- badges: true
- comments: true
- author: 최규빈

### import 

In [1]:
from fastai.data.all import *
from fastai.vision.all import * 

### Path

`-` 기능: 현재폴더, 혹은 그 하위폴더들에 속한 파일의 목록을 볼 수 있다. 

In [2]:
path=Path() # Path클래스에서 인스턴스생성 

In [3]:
(path/'ghtop_images').ls()

(#2) [Path('ghtop_images/token.png'),Path('ghtop_images/sparknb.gif')]

`-` Path(...)에서 ...에 무엇을 넣느냐에 따라 원하는 경로를 설정할 수 있다. 

In [4]:
path=Path('/home')

In [5]:
path.ls()

(#1) [Path('/home/cgb4')]

`-` 폴더를 만들수 있다. 

In [6]:
path=Path() 

In [7]:
(path/'asdf').mkdir()

In [8]:
(path/'asdf').ls()

(#0) []

`-` 이미 폴더가 존재할 때는 아래와 같이 에러가 발생

In [9]:
(path/'asdf').mkdir()

FileExistsError: [Errno 17] File exists: 'asdf'

In [10]:
(path/'asdf').mkdir(exist_ok=True)

`-` 생성한 폴더를 지우는 방법 

In [11]:
(path/'asdf').rmdir()

### import some functions 

`-` ref: https://github.com/fastai/fastbook/blob/master/utils.py

In [12]:
def search_images_ddg(key,max_n=200):
    """Search for 'key' with DuckDuckGo and return a unique urls of 'max_n' images
       (Adopted from https://github.com/deepanprabhu/duckduckgo-images-api)
    """
    url        = 'https://duckduckgo.com/'
    params     = {'q':key}
    res        = requests.post(url,data=params)
    searchObj  = re.search(r'vqd=([\d-]+)\&',res.text)
    if not searchObj: print('Token Parsing Failed !'); return
    requestUrl = url + 'i.js'
    headers    = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:71.0) Gecko/20100101 Firefox/71.0'}
    params     = (('l','us-en'),('o','json'),('q',key),('vqd',searchObj.group(1)),('f',',,,'),('p','1'),('v7exp','a'))
    urls       = []
    while True:
        try:
            res  = requests.get(requestUrl,headers=headers,params=params)
            data = json.loads(res.text)
            for obj in data['results']:
                urls.append(obj['image'])
                max_n = max_n - 1
                if max_n < 1: return L(set(urls))     # dedupe
            if 'next' not in data: return L(set(urls))
            requestUrl = url + data['next']
        except:
            pass

- 검색어를 입력하면 url을 반환해주는 함수 

### 이미지 크롤링 

`-` 크롤링은 (1) 검색 (2) 이미지주소를 찾음 (3) 해당주소로 이동하여 저장하는 과정을 반복하면 된다. 

`-` 주소들의 리스트는 search_images_ddg() 함수를 사용하여 얻을 수 있다. 

In [13]:
search_images_ddg('iu',max_n=10)

(#1) ['https://upload.wikimedia.org/wikipedia/commons/3/33/IU_at_%22Midnight_Runners%22_VIP_premiere%2C_7_August_2017_03.jpg']

`-` url을 이용하여 이미지를 저장하자. 

In [16]:
download_images(dest=Path(),urls=search_images_ddg('iu',max_n=1))

In [14]:
download_images?

Signature:
download_images(
    dest,
    url_file=None,
    urls=None,
    max_pics=1000,
    n_workers=8,
    timeout=4,
    preserve_filename=False,
)
Docstring: Download images listed in text file `url_file` to path `dest`, at most `max_pics`
File:      ~/anaconda3/envs/bda2021/lib/python3.8/site-packages/fastai/vision/utils.py
Type:      function
